In [1]:
import h5py
import os
import hdf5plugin
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
data_path = os.environ['DATA_DIR']
f_name = 'train_multi_inputs'

from_path = Path(data_path + f_name + '.h5')
to_path = Path(data_path + f_name + '_T.h5')

In [3]:
# transform and copy main chunk
with h5py.File(from_path, 'r') as from_file, h5py.File(to_path, 'w') as to_file:
    # copy index and columnnames
    in_and_col_names = [('axis0', 'accessibility'), ('axis1', 'cell')]
    for ds_old, ds_new in in_and_col_names:
        from_dset = from_file[f_name + '/' + ds_old]
        to_dset = to_file.create_dataset(ds_new, from_dset.shape, dtype=from_dset.dtype, compression="gzip")
        to_dset[:] = from_dset[:]

    # create dataset objects
    from_dset = from_file[f_name + '/block0_values']
    to_dset = to_file.create_dataset('values', from_dset.shape[::-1], dtype=from_dset.dtype, compression="gzip")

    # iterate over old dataset
    batchsize = 40000
    iterations = int(np.ceil(from_dset.shape[1] / batchsize))
    
    for i in tqdm(range(iterations)):
        to_dset[i*batchsize:(i+1)*batchsize, :] = from_dset[:, i*batchsize: (i+1)*batchsize].T

  0%|          | 0/6 [00:00<?, ?it/s]